In [1]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# model, tokenizer

In [2]:
additional_tokens = {'additional_special_tokens': ['[learn1]', '[learn2]', '[learn3]', '[learn4]', '[learn5]', '[learn6]']}

In [3]:
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print_trainable_parameters(model)

trainable params: 124439808 || all params: 124439808 || trainable%: 100.0


In [5]:
num_added_toks = tokenizer.add_special_tokens(additional_tokens)

In [6]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50265, 768)

In [7]:
# save the tokenizer

tokenizer.save_pretrained('DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer')

('DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer/tokenizer_config.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer/special_tokens_map.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer/vocab.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer/merges.txt',
 'DocRED/GPT_w_ner_short_relation_RCfirst/gpt2_tokenizer/added_tokens.json')

# data

In [8]:
# load the json file from DocRED/data/test.json and DocRED/data/rel_info.json

import json

with open('DocRED/data/train_annotated.json') as f:
    train_set = json.load(f)


with open('DocRED/data/rel_info.json') as f:
    rel_info = json.load(f)

In [9]:
train_set[0]

{'vertexSet': [[{'pos': [0, 4],
    'type': 'ORG',
    'sent_id': 0,
    'name': 'Zest Airways, Inc.'},
   {'sent_id': 0,
    'type': 'ORG',
    'pos': [10, 15],
    'name': 'Asian Spirit and Zest Air'},
   {'name': 'AirAsia Zest', 'pos': [6, 8], 'sent_id': 0, 'type': 'ORG'},
   {'name': 'AirAsia Zest', 'pos': [19, 21], 'sent_id': 6, 'type': 'ORG'}],
  [{'name': 'Ninoy Aquino International Airport',
    'pos': [4, 8],
    'sent_id': 3,
    'type': 'LOC'},
   {'name': 'Ninoy Aquino International Airport',
    'pos': [26, 30],
    'sent_id': 0,
    'type': 'LOC'}],
  [{'name': 'Pasay City', 'pos': [31, 33], 'sent_id': 0, 'type': 'LOC'}],
  [{'name': 'Metro Manila', 'pos': [34, 36], 'sent_id': 0, 'type': 'LOC'}],
  [{'name': 'Philippines', 'pos': [38, 39], 'sent_id': 0, 'type': 'LOC'},
   {'name': 'Philippines', 'pos': [13, 14], 'sent_id': 4, 'type': 'LOC'},
   {'sent_id': 5,
    'type': 'LOC',
    'pos': [25, 29],
    'name': 'Republic of the Philippines'}],
  [{'name': 'Manila', 'pos': 

In [10]:
"""
the names of vertextSet can be the same, but the pos should be different
structure:
'vertexSet': 
    [
        (for the same entity but in different synonyms and different sentences)
        [
            {
                'pos':[start, end],
                'type': 'NER',
                'sent_id': 0,
                'name': 'string',
            },
            {}
        ],
        [entity-2]
    ]
'labels':
    [
        {
            'r': 'Pxx',
            'h': 0,
            't': 1,
            'evidence': [2, 3, 4],
        },
        {}
    ]
'title': 'string',
'sents':
    [
        ['word0', 'word1',]
        ['word0', 'word1',]
    ]
"""

"\nthe names of vertextSet can be the same, but the pos should be different\nstructure:\n'vertexSet': \n    [\n        {\n            'pos':[start, end],\n            'type': 'NER',\n            'sent_id': 0,\n            'name': 'string',\n        },\n        {}\n    ]\n'labels':\n    [\n        {\n            'r': 'Pxx',\n            'h': 0,\n            't': 1,\n            'evidence': [2, 3, 4],\n        },\n        {}\n    ]\n'title': 'string',\n'sents':\n    [\n        ['word0', 'word1',]\n        ['word0', 'word1',]\n    ]\n"

In [89]:
with open('DocRED/data/ner_info.json') as f:
    ner_info = json.load(f)

with open('DocRED/data/rel_info.json') as f:
    relation_info = json.load(f)

In [12]:
"""# doc-level is too long for gpt-2, so we need to split the doc-level into bi-sent-level

relation_dict = {
    'id': [],
    'text':[],
    'entity': [],
    'relation': []
}

for i in range(len(train_set)):
    # id
    relation_dict['id'].append(i)

    # text
    sents = ""
    for sent in train_set[i]['sents']:
        # flatten the sent list
        a = " ".join(sent)
        sents += a.lower() + " "
    # if there are space, delete the first and last space of the sents
    sents = sents.strip()
    # delete double space in the sents
    sents = sents.replace("  ", " ")
    relation_dict['text'].append(sents)
    del sents

    # entity
    entity = []
    entity_list = []
    entity_flat = {}
    entity_count = 0
    for sent_item in train_set[i]['vertexSet']:
        for item in sent_item:
            entity_item = []
            if item['name'].lower() not in entity_list:
                entity_list.append(item['name'].lower().strip())
                entity_item.append(item['name'].lower().strip())
                entity_item.append(ner_info[item['type']])

                entity.append(entity_item)
            
            # add the entity_flat
            entity_flat[entity_count] = item['name'].lower().strip()
            entity_count += 1

    # release the entity_list and entity_item
    del entity_item
    del entity_count
        

    # relation pairs
    relation_pairs = {}
    for relation_item in train_set[i]['labels']:
        pair = []
        head = entity_flat[relation_item['h']]
        tail = entity_flat[relation_item['t']]
        pair.append(head)
        pair.append(tail)

        relation  = relation_info[relation_item['r']]
        if relation not in relation_pairs.keys():
            relation_pairs[relation] = []

        relation_pairs[relation].append(pair)
    del pair
    del head
    del tail

    # add the entity and relation pairs to the relation_dict
    relation_dict['entity'].append(entity)
    relation_dict['relation'].append(relation_pairs)
    break


# save the relation_dict to a json file

# with open('DocRED/data/DocRED_baseline_metadata/relation_dict.json', 'w') as f:
#     json.dump(relation_dict, f)"""

In [91]:
relation_dict = {
    'text':[],
    'entity': [],
    'relation': []
}
id_count = 0

for i in range(len(train_set)):
    

    # text
    sent_pairs = []
    for sent_index, sent in enumerate(train_set[i]['sents']):
        sents = ""

        # flatten the sent list
        a = " ".join(sent)
        sents += a.lower() + " "

        # and the next_sent if it exists
        try:
            next_sent = train_set[i]['sents'][sent_index + 1]
            b = " ".join(next_sent)
            sents += b.lower() + " "
        except:
            pass
        # post process the sents for some spaces
        sents = sents.strip()
        sents = sents.replace("  ", " ")

        relation_dict['text'].append(sents)
            
    del sents


    # entity
    entity = []
    for index in range(len(train_set[i]['sents'])):
        # focus on the current sent and the next sent if it exists
        if index + 1 < len(train_set[i]['sents']):
            next_index = index + 1
        else:
            next_index = index
        # group the entities for every 2 sents, no repeated entities in one group
        c_sent_entity_lists = []
        next_sent_entity_lists = []
        entity_for_each_2_sents = []

        for entity_spans in train_set[i]['vertexSet']:
            for item in entity_spans:
                entity_item = []
                # if neither in the current sent nor in the next sent, continue
                if item['sent_id'] != index and item['sent_id'] != next_index:
                    continue
                # also store the first pos of the entity in the entity_item
                # it will look like this: [[entity_name, sent_index, pos1, ner_type]]
                entity_item = [item['name'].lower().strip(), item['sent_id'], item['pos'][0], ner_info[item['type']]]

                if entity_item[1] == index:
                    c_sent_entity_lists.append(entity_item)
                else:
                    next_sent_entity_lists.append(entity_item)

        # sort the c_sent_entity_lists and next_sent_entity_lists by the pos in ascending order
        c_sent_entity_lists.sort(key=lambda x: x[2])
        if index != next_index:
            next_sent_entity_lists.sort(key=lambda x: x[2])
        
        entity_list = []
        for item in c_sent_entity_lists:
            if item[0] not in entity_list:
                entity_list.append(item[0])
                entity_for_each_2_sents.append([item[0], item[3]])
            
        if index != next_index:
            for item in next_sent_entity_lists:
                if item[0] not in entity_list:
                    entity_list.append(item[0])
                    entity_for_each_2_sents.append([item[0], item[3]])

        relation_dict['entity'].append(entity_for_each_2_sents)

    del entity_item
    del c_sent_entity_lists
    del next_sent_entity_lists
    del entity_for_each_2_sents
        

    # relation pairs
    relation_pairs = []

    for index in range(len(train_set[i]['sents'])):
        relation_pairs_for_each_2_sents = {}
        # focus on the current sent and the next sent if it exists
        if index + 1 < len(train_set[i]['vertexSet']):
            next_index = index + 1
        else:
            next_index = index

        # heads, tails: ['entity_name', start_pos]
        for relation_item in train_set[i]['labels']:
            heads = []
            tails = []
            
            # head
            head_exist = False
            for head_span in train_set[i]['vertexSet'][relation_item['h']]:
                if head_span['sent_id'] == index or head_span['sent_id'] == next_index:
                    heads.append([head_span['name'].lower().strip(), head_span['pos'][0]])
                    head_exist = True
            if not head_exist:
                continue
    
            # tail
            tail_exist = False
            for tail_span in train_set[i]['vertexSet'][relation_item['t']]:
                if tail_span['sent_id'] == index or tail_span['sent_id'] == next_index:
                    tails.append([tail_span['name'].lower().strip(), tail_span['pos'][0]])
                    tail_exist = True
            if not tail_exist:
                continue
            

            if relation_info[relation_item['r']] not in relation_pairs_for_each_2_sents.keys():
                relation_pairs_for_each_2_sents[relation_info[relation_item['r']]] = []
            for head in heads:
                for tail in tails:
                    relation_pairs_for_each_2_sents[relation_info[relation_item['r']]].append([head[0], tail[0]])

        relation_dict['relation'].append(relation_pairs_for_each_2_sents)


    
    # break


# save the relation_dict to a json file

# with open('DocRED/data/bi-sent-pre-process.json', 'w') as f:
    # json.dump(relation_dict, f)

In [65]:
len(relation_dict['text']) == len(relation_dict['entity']) == len(relation_dict['relation'])

True

In [98]:
relation_dict['relation'][0]

{'headquarters location': [['zest airways, inc.', 'pasay city'],
  ['asian spirit and zest air', 'pasay city'],
  ['airasia zest', 'pasay city']],
 'country': [['zest airways, inc.', 'philippines'],
  ['asian spirit and zest air', 'philippines'],
  ['airasia zest', 'philippines'],
  ['pasay city', 'philippines'],
  ['manila', 'philippines'],
  ['metro manila', 'philippines'],
  ['ninoy aquino international airport', 'philippines']],
 'located in the administrative territorial entity': [['pasay city',
   'metro manila'],
  ['metro manila', 'philippines'],
  ['ninoy aquino international airport', 'pasay city']],
 'contains administrative territorial entity': [['philippines',
   'metro manila'],
  ['metro manila', 'pasay city']]}

In [9]:
import json

ner = 1
from datasets import Dataset

relation_dict = {}
if ner:
    with open('DocRED/data/bi-sent-pre-process.json') as f:
        relation_dict = json.load(f)

    dataset = Dataset.from_dict(
        {
            'text': relation_dict['text'],
            'entity': relation_dict['entity'],
            'relation': relation_dict['relation']
        }
    )

else:
    pass

In [10]:
dataset

Dataset({
    features: ['text', 'entity', 'relation'],
    num_rows: 24256
})

In [11]:
dataset[0]

{'text': 'zest airways , inc. operated as airasia zest ( formerly asian spirit and zest air ) , was a low - cost airline based at the ninoy aquino international airport in pasay city , metro manila in the philippines . it operated scheduled domestic and international tourist services , mainly feeder services linking manila and cebu with 24 domestic destinations in support of the trunk route operations of other airlines .',
 'entity': [['zest airways, inc.', 'organization'],
  ['airasia zest', 'organization'],
  ['asian spirit and zest air', 'organization'],
  ['ninoy aquino international airport', 'location'],
  ['pasay city', 'location'],
  ['metro manila', 'location'],
  ['philippines', 'location'],
  ['manila', 'location'],
  ['cebu', 'location'],
  ['24', 'number']],
 'relation': {'applies to jurisdiction': None,
  'author': None,
  'award received': None,
  'basin country': None,
  'capital': None,
  'capital of': None,
  'cast member': None,
  'chairperson': None,
  'characters':

In [12]:
len(dataset['entity'][0])

10

In [13]:
"""relation_info_dict = {}
for id, relation in enumerate(dataset[0]['relation'].keys()):
    relation_info_dict[relation] = id

with open('DocRED/data/relation-index.json', 'w') as f:
    json.dump(relation_info_dict, f)"""

with open('DocRED/data/relation-index.json') as f:
    relation_info_dict = json.load(f)

In [15]:
def pro_processing_ner(example, tokenizer, padding=True):
    texts = example['text']
    input_texts = []
    for index in range(len(texts)):
        # # entity extraction and NER
        text = texts[index].lower().strip() + " [learn1] [learn2]"
        # for entity in example['entity'][index]:
        #     text = text + " entity : " + entity[0] + " , type : " + entity[1] + " ;"
        # text = text[:-1] + "."
        # # print("1")
        # add relation classificaiton
        # text = text.lower().strip() + " [learn3] [learn4]"
        for relation_type, relation_pair in example['relation'][index].items():
            if relation_pair:
                # relation classification
                text_w_relation = text + " for the relation " + relation_type + " : 1 ."

                # related entity extraction and NER, order by the apperance
                text_w_relation = text_w_relation.lower().strip() + " [learn3] [learn4]"
                text_w_relation = text_w_relation + " and the entities related to the relation " + relation_type + " are :"
                relation_entities = []
                for pair in relation_pair:
                    if pair[0] not in relation_entities:
                        relation_entities.append(pair[0])
                    if pair[1] not in relation_entities:
                        relation_entities.append(pair[1])
                for entity in example['entity'][index]:
                    if entity[0] in relation_entities:
                        text_w_relation = text_w_relation + " " + entity[0] + " , type : " + entity[1] + " ;"
                text_w_relation = text_w_relation[:-1] + "."

                # head and tail entity
                text_w_relation = text_w_relation.lower().strip() + " [learn5] [learn6] the entity paris are :"
                for pair in relation_pair:
                    text_w_relation = text_w_relation + " head entity: " + pair[0] + " , tail entity: " + pair[1] + ";"
                text_w_relation = text_w_relation[:-1] + "." + tokenizer.eos_token
                input_texts.append(text_w_relation)

            else:
                text_w_relation = text + " for the relation " + relation_type + " : 0 ." + tokenizer.eos_token
                input_texts.append(text_w_relation)

    return {
        'input_ids': input_texts
        }

In [16]:
# feed the dataset:dataset to the pro_processing_ner() function with tokenizer, at each time, we feed 30 examples to the function, and then save the output to a json file
# each time the return of the function is a dict, we need to save the dict to a list, and then save the list to a json file

import json
from tqdm import tqdm


output = {"input_texts": []}

for i in tqdm(range(0, len(dataset), 30)):
    result = pro_processing_ner(dataset[i:i+30], tokenizer)
    output["input_texts"].extend(result["input_ids"])

100%|██████████| 809/809 [00:12<00:00, 63.66it/s]


In [20]:
output["input_texts"][12]

'zest airways , inc. operated as airasia zest ( formerly asian spirit and zest air ) , was a low - cost airline based at the ninoy aquino international airport in pasay city , metro manila in the philippines . it operated scheduled domestic and international tourist services , mainly feeder services linking manila and cebu with 24 domestic destinations in support of the trunk route operations of other airlines . [learn1] [learn2] for the relation contains administrative territorial entity : 1 . [learn3] [learn4] and the entities related to the relation contains administrative territorial entity are : pasay city , type : location ; metro manila , type : location ; philippines , type : location . [learn5] [learn6] the entity paris are : head entity: philippines , tail entity: metro manila; head entity: metro manila , tail entity: pasay city.<|endoftext|>'

In [17]:
len(output['input_texts'])

2328576

In [21]:
# make the output["input_texts"] into a dataset
from datasets import Dataset

input_text_dataset = Dataset.from_dict(
    {
        'input_texts': output['input_texts'],
    }
)


In [22]:
tokenized_dataset = input_text_dataset.map(lambda example: tokenizer(example['input_texts'], padding='max_length', truncation=True, max_length=1024, pad_to_max_length=True), batched=True)

Map:   0%|          | 0/2328576 [00:00<?, ? examples/s]

In [23]:
# remove the column of input_texts in the tokenized_dataset
tokenized_dataset.remove_columns('input_texts')

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2328576
})

In [24]:
# save the tokenized_dataset

tokenized_dataset.save_to_disk('DocRED/GPT_w_ner_short_relation_RCfirst/train_data_ner_short_relation_RCfirst')
# with open('DocRED/data/train_ner_short_relation.json', 'w') as f:
#     json.dump(tokenized_dataset, f)

Saving the dataset (0/26 shards):   0%|          | 0/2328576 [00:00<?, ? examples/s]

In [27]:


from datasets import Dataset

tokenized_dataset = Dataset.load_from_disk('DocRED/GPT_w_ner_short_relation_RCfirst/train_data_ner_short_relation_RCfirst')
tokenized_dataset = tokenized_dataset.remove_columns('input_texts')

In [28]:
# only take the first len(tokenized_dataset) // 50 examples to train the model

tokenized_dataset = tokenized_dataset.select(range(len(tokenized_dataset) // 50))

In [29]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [30]:
tokenizer.decode(tokenized_dataset[66]['input_ids'])

'zest airways, inc. operated as airasia zest ( formerly asian spirit and zest air ), was a low - cost airline based at the ninoy aquino international airport in pasay city, metro manila in the philippines. it operated scheduled domestic and international tourist services, mainly feeder services linking manila and cebu with 24 domestic destinations in support of the trunk route operations of other airlines. [learn1] [learn2] for the relation participant of : 0.<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|

In [31]:
tokenized_dataset.__getitems__([1,4])

[{'input_ids': tensor([   89,   395,  1633,  ..., 50258, 50258, 50258]),
  'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])},
 {'input_ids': tensor([   89,   395,  1633,  ..., 50258, 50258, 50258]),
  'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}]

# trainer

In [32]:
import wandb

wandb.init(
    # set the wandb project where this run will be logged
    project="GPT2-intermediate",
    # notes="PubmedBERT-FT-NER_w_NERin_10epochs",
    name="GPT2-short_relation_RCfirst_DocRED-w-ner-5epochs"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 309439737 (tian1995). Use `wandb login --relogin` to force relogin


In [34]:
import transformers
from transformers import DataCollatorForLanguageModeling

trainer = transformers.Trainer(
    model=model, 
    train_dataset=tokenized_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, 
        gradient_accumulation_steps=2,
        warmup_steps=1000, 
        num_train_epochs=5,
        learning_rate=2e-4, 
        # fp16=True,
        logging_steps=100, 
        report_to="wandb",
        save_strategy="epoch",
        output_dir='DocRED/GPT_w_ner_short_relation_RCfirst'
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [35]:
trainer.train()

/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/58215 [00:00<?, ?it/s]

{'loss': 33.8534, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 4.2144, 'learning_rate': 4e-05, 'epoch': 0.02}
{'loss': 3.2271, 'learning_rate': 6e-05, 'epoch': 0.03}
{'loss': 2.6751, 'learning_rate': 8e-05, 'epoch': 0.03}
{'loss': 2.1585, 'learning_rate': 0.0001, 'epoch': 0.04}
{'loss': 1.7174, 'learning_rate': 0.00012, 'epoch': 0.05}
{'loss': 1.274, 'learning_rate': 0.00014, 'epoch': 0.06}
{'loss': 0.9785, 'learning_rate': 0.00016, 'epoch': 0.07}
{'loss': 0.7323, 'learning_rate': 0.00018, 'epoch': 0.08}
{'loss': 0.6763, 'learning_rate': 0.0002, 'epoch': 0.09}
{'loss': 0.5325, 'learning_rate': 0.00019965044131783623, 'epoch': 0.09}
{'loss': 0.4464, 'learning_rate': 0.00019930088263567247, 'epoch': 0.1}
{'loss': 0.4051, 'learning_rate': 0.0001989513239535087, 'epoch': 0.11}
{'loss': 0.3537, 'learning_rate': 0.00019860176527134493, 'epoch': 0.12}
{'loss': 0.3062, 'learning_rate': 0.00019825220658918117, 'epoch': 0.13}
{'loss': 0.2958, 'learning_rate': 0.00019790264790701742, 'epoch': 

TrainOutput(global_step=58215, training_loss=0.21307460664627756, metrics={'train_runtime': 49078.7082, 'train_samples_per_second': 4.745, 'train_steps_per_second': 1.186, 'train_loss': 0.21307460664627756, 'epoch': 5.0})

In [36]:
wandb.finish()
trainer.save_model("DocRED/GPT_w_ner_short_relation_RCfirst")

# save the tokenizer
tokenizer.save_pretrained("DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer")

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,5.0
train/global_step,58215


('DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer/tokenizer_config.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer/special_tokens_map.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer/vocab.json',
 'DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer/merges.txt',
 'DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer/added_tokens.json')

# Inference

In [37]:
from transformers import AutoModelForCausalLM

checkpoint = "DocRED/GPT_w_ner_short_relation_RCfirst"

In [38]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("DocRED/GPT_w_ner_short_relation_RCfirst/tokenizer")
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [39]:
# output all of the special tokens in the tokenizer
tokenizer.all_special_tokens

['<|startoftext|>',
 '<|endoftext|>',
 '<|pad|>',
 '[learn1]',
 '[learn2]',
 '[learn3]',
 '[learn4]',
 '[learn5]',
 '[learn6]']

In [40]:
import torch

model.eval()
model.to("cpu")
# inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. [learn1] [learn2] entity :", return_tensors="pt")

inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again.", return_tensors="pt", padding='max_length', max_length=1000)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=20, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0])

/home/tian/mambaforge/envs/BioRED/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad

test data pre-processing

In [41]:
import json

from datasets import Dataset

ner = 1

test_relation_dict = {}
if ner:
    with open('DocRED/data/bi-sent-pre-process_test.json') as f:
        test_relation_dict = json.load(f)

    test_dataset = Dataset.from_dict(
        {
            'text': test_relation_dict['text'],
            'entity': test_relation_dict['entity'],
            'relation': test_relation_dict['relation']
        }
    )

else:
    pass


dataset = test_dataset

In [42]:
def pro_processing_ner(example, tokenizer, padding=True):
    texts = example['text']
    input_texts = []
    for index in range(len(texts)):
        # # entity extraction and NER
        text = texts[index].lower().strip() + " [learn1] [learn2]"
        # for entity in example['entity'][index]:
        #     text = text + " entity : " + entity[0] + " , type : " + entity[1] + " ;"
        # text = text[:-1] + "."
        # # print("1")
        # add relation classificaiton
        # text = text.lower().strip() + " [learn3] [learn4]"
        for relation_type, relation_pair in example['relation'][index].items():
            if relation_pair:
                # relation classification
                text_w_relation = text + " for the relation " + relation_type + " : 1 ."

                # related entity extraction and NER, order by the apperance
                text_w_relation = text_w_relation.lower().strip() + " [learn3] [learn4]"
                text_w_relation = text_w_relation + " and the entities related to the relation " + relation_type + " are :"
                relation_entities = []
                for pair in relation_pair:
                    if pair[0] not in relation_entities:
                        relation_entities.append(pair[0])
                    if pair[1] not in relation_entities:
                        relation_entities.append(pair[1])
                for entity in example['entity'][index]:
                    if entity[0] in relation_entities:
                        text_w_relation = text_w_relation + " " + entity[0] + " , type : " + entity[1] + " ;"
                text_w_relation = text_w_relation[:-1] + "."

                # head and tail entity
                text_w_relation = text_w_relation.lower().strip() + " [learn5] [learn6] the entity paris are :"
                for pair in relation_pair:
                    text_w_relation = text_w_relation + " head entity: " + pair[0] + " , tail entity: " + pair[1] + ";"
                text_w_relation = text_w_relation[:-1] + "." + tokenizer.eos_token
                input_texts.append(text_w_relation)

            else:
                text_w_relation = text + " for the relation " + relation_type + " : 0 ." + tokenizer.eos_token
                input_texts.append(text_w_relation)

    return {
        'input_ids': input_texts
        }

import json
from tqdm import tqdm


output = {"input_texts": []}

for i in tqdm(range(0, len(dataset), 30)):
    result = pro_processing_ner(dataset[i:i+30], tokenizer)
    output["input_texts"].extend(result["input_ids"])


from datasets import Dataset

input_text_dataset = Dataset.from_dict(
    {
        'input_texts': output['input_texts'],
    }
)


100%|██████████| 269/269 [00:04<00:00, 64.43it/s]


In [43]:
tokenized_dataset = input_text_dataset.map(lambda example: tokenizer(example['input_texts'], padding='max_length', truncation=True, max_length=1024, pad_to_max_length=True), batched=True)

tokenized_dataset.remove_columns('input_texts')

tokenized_dataset.save_to_disk('DocRED/GPT_w_ner_short_relation_RCfirst/test_data_ner_short_relation')
# with open('DocRED/data/train_ner_short_relation.json', 'w') as f:
#     json.dump(tokenized_dataset, f)



Map:   0%|          | 0/773472 [00:00<?, ? examples/s]

Saving the dataset (0/9 shards):   0%|          | 0/773472 [00:00<?, ? examples/s]

randomly select test data

In [44]:
from datasets import Dataset

tokenized_test_dataset = Dataset.load_from_disk('DocRED/GPT_w_ner_short_relation_RCfirst/test_data_ner_short_relation')
tokenized_test_dataset = tokenized_test_dataset.remove_columns('input_texts')

tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [45]:
input_text_dataset[288]

{'input_texts': 'besides digital terrestrial transmission , it is available on the subscription - based encrypted services of nova and cosmote tv . skai tv is also a member of digea , a consortium of private television networks introducing digital terrestrial transmission in greece . [learn1] [learn2] for the relation applies to jurisdiction : 0 .<|endoftext|>'}

In [46]:
len(tokenized_dataset) / 96

8057.0

In [69]:
# random sample 1 example from the test_dataset
import random
# have a random seed
random.seed(60)

# random_test_index = random.randint(0, 8056)
random_test_index = 30
print("index: ", random_test_index * 96, " - ", random_test_index * 96 + 95)
print("index: ", random_test_index)
# print(tokenizer.decode(tokenized_test_dataset[random_test_index * 96]['input_ids']))

# output the length of tokenized_test_dataset[index]['input_ids'] except the padding tokens. the tokenized_test_dataset[index]['input_ids'] is tensor

input_ids_lists = tokenized_test_dataset[random_test_index * 96 : random_test_index * 96 + 96]['input_ids'].tolist()

all_actually_inputs = {"learn2_index": [], "valid_length": []}
all_gold_truths = []

for input_ids_list in tqdm(input_ids_lists):
    
    print(tokenizer.decode(input_ids_list))

    valid_length = len(input_ids_list) - input_ids_list.count(tokenizer.pad_token_id)

    # print(tokenizer.decode(tokenized_test_dataset[index]['input_ids'][:valid_length]))

    # generate a lower triangle matrix of 1s with the shape is (valid_length, valid_length), using torch

    low_triangle_matrix = torch.tril(torch.ones((valid_length, valid_length), dtype=torch.long))

    # find the index of the token id of "[learn2]" in the tokenized_test_dataset[index]['input_ids'] tensor

    learn2_index = input_ids_list.index(tokenizer.convert_tokens_to_ids("[learn2]"))

    # have a vector to store the token of okenized_test_dataset[index]['input_ids'][learn2_index + 1:valid_length]

    gold_truth = input_ids_list[learn2_index + 1:valid_length]

    all_actually_inputs["learn2_index"].append(learn2_index)

    all_actually_inputs["valid_length"].append(valid_length)

    all_gold_truths.append(gold_truth)



index:  2880  -  2975
index:  30


  0%|          | 0/96 [00:00<?, ?it/s]

conrad oberon johnson ( november 15, 1915 – february 3, 2008 ) was an american music educator, long associated with the city of houston, who was inducted into the texas bandmasters hall of fame in 2000. born in victoria, texas, conrad johnson was nine when his family moved to houston. [learn1] [learn2] for the relation applies to jurisdiction : 0.<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad

quick inference

In [70]:
from tqdm.notebook import trange, tqdm
import torch
import numpy as np


model.eval()
outputs = []
model.to("cuda")

batch_output = []
all_batch_output = []
all_accuracy = []


with torch.no_grad():
    # feed the actually_input to the model by 10 examples each time
    for input_index, input_ids_list in enumerate(input_ids_lists):
        print(input_index + 1, " / 96")
        for i in range(all_actually_inputs['learn2_index'][input_index] + 1, all_actually_inputs['valid_length'][input_index]):
            output = model(input_ids=torch.tensor(input_ids_list[:i]).to("cuda"))
            current_output = np.array(output['logits'].cpu())
            max_index = np.argmax(current_output[-1, :], axis=0)
            batch_output.append(max_index)
            # break
        if len(batch_output) == len(all_gold_truths[input_index]):
            accuracy = sum(np.array(batch_output) == np.array(all_gold_truths[input_index])) / len(batch_output)
            all_accuracy.append(accuracy)
            all_batch_output.append(batch_output)
            batch_output = []
        else:
            print("error")
        
    # print(tokenizer.batch_decode(max_index, skip_special_tokens=False)[0])

1  / 96
2  / 96
3  / 96
4  / 96
5  / 96
6  / 96
7  / 96
8  / 96
9  / 96
10  / 96
11  / 96
12  / 96
13  / 96
14  / 96
15  / 96
16  / 96
17  / 96
18  / 96
19  / 96
20  / 96
21  / 96
22  / 96
23  / 96
24  / 96
25  / 96
26  / 96
27  / 96
28  / 96
29  / 96
30  / 96
31  / 96
32  / 96
33  / 96
34  / 96
35  / 96
36  / 96
37  / 96
38  / 96
39  / 96
40  / 96
41  / 96
42  / 96
43  / 96
44  / 96
45  / 96
46  / 96
47  / 96
48  / 96
49  / 96
50  / 96
51  / 96
52  / 96
53  / 96
54  / 96
55  / 96
56  / 96
57  / 96
58  / 96
59  / 96
60  / 96
61  / 96
62  / 96
63  / 96
64  / 96
65  / 96
66  / 96
67  / 96
68  / 96
69  / 96
70  / 96
71  / 96
72  / 96
73  / 96
74  / 96
75  / 96
76  / 96
77  / 96
78  / 96
79  / 96
80  / 96
81  / 96
82  / 96
83  / 96
84  / 96
85  / 96
86  / 96
87  / 96
88  / 96
89  / 96
90  / 96
91  / 96
92  / 96
93  / 96
94  / 96
95  / 96
96  / 96


In [78]:
# have the average accuracy
if len(all_accuracy) == 96:

    print(sum(all_accuracy) / len(all_accuracy))

0.8736544078577281


In [77]:
# have the max accuracy and the index of the max accuracy

max_accuracy = max(all_accuracy)
max_accuracy_index = all_accuracy.index(max_accuracy)

In [76]:
# output the index of the correct prediction

for index, item in enumerate(zip(all_batch_output[max_accuracy_index], all_gold_truths[max_accuracy_index])):
    if item[0] == item[1]:
        print(index, ": ", tokenizer.decode(item[0]))

0 :  for
1 :   the
2 :   relation
3 :   member
4 :   of
5 :   sports
6 :   team
7 :   :
8 :   0
9 :  .
10 :  <|endoftext|>


In [75]:
tokenizer.decode(input_ids_lists[max_accuracy_index])

'conrad oberon johnson ( november 15, 1915 – february 3, 2008 ) was an american music educator, long associated with the city of houston, who was inducted into the texas bandmasters hall of fame in 2000. born in victoria, texas, conrad johnson was nine when his family moved to houston. [learn1] [learn2] for the relation member of sports team : 0.<|endoftext|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|

whole inference

In [79]:
import json

def for_the_relation (index, input,tokenizer=tokenizer):
    rel_info = {}
    with open('DocRED/data/relation-index.json') as f:
        rel_info = json.load(f)
    rel_info_list = [relation for relation in rel_info.keys()]
    relation_input = " for the relation " + rel_info_list[index].lower() + " : "
    tokenized_relation_input = tokenizer.encode(relation_input, add_special_tokens=False, return_tensors="pt")
    # print("before relation classification: ", tokenizer.decode(input))
    return (torch.cat((input, tokenized_relation_input[0]), dim=0))


def and_the_entities_related (index, input,tokenizer=tokenizer):
    rel_info = {}
    with open('DocRED/data/relation-index.json') as f:
        rel_info = json.load(f)
    rel_info_list = [relation for relation in rel_info.keys()]
    relation_input =  "and the entities related to the relation " + rel_info_list[index].lower() + " are : "
    tokenized_relation_input = tokenizer.encode(relation_input, add_special_tokens=False, return_tensors="pt")
    # print("before relation classification: ", tokenizer.decode(input))
    return (torch.cat((input, tokenized_relation_input[0]), dim=0))


In [80]:
from tqdm.notebook import trange, tqdm
import torch
import numpy as np


model.eval()
outputs = []
model.to("cuda")
output_texts = []




with torch.no_grad():
    # feed the actually_input to the model by 10 examples each time
    for input_index, input_ids_list in enumerate(input_ids_lists):
        print(input_index + 1, " / 96")
        start = all_actually_inputs['learn2_index'][input_index] + 1
        input_ids = torch.tensor(input_ids_list[:start]).to("cuda")
        relation_classfication = False
        relation_extraction = False

        input_ids = for_the_relation(input_index, input_ids.to("cpu"))
        input_ids = input_ids.to("cuda")
        relation_classfication = True

        while((input_ids[-1].item() != tokenizer.eos_token_id) and (len(input_ids) < 1024)):
            output = model(input_ids=input_ids)
            current_output = np.array(output['logits'].cpu())
            max_index = np.argmax(current_output[-1, :], axis=0)

            if max_index == tokenizer.convert_tokens_to_ids("[learn4]") and (not relation_classfication):
                input_ids = torch.cat((input_ids, torch.tensor(max_index).unsqueeze(0).to("cuda")), dim=0)
                input_ids = for_the_relation(input_index, input_ids.to("cpu"))
                input_ids = input_ids.to("cuda")
                relation_classfication = True
                continue

            if relation_classfication and (not relation_extraction):

                # find the possibilities for token id tokenizer.conver_token_to_id("0") and tokenizer.conver_token_to_id("1") in the current_output

                zero_index = tokenizer.convert_tokens_to_ids("0")
                one_index = tokenizer.convert_tokens_to_ids("1")
                zero_possibility = current_output[-1, zero_index]
                print("zero_possibility: ", zero_possibility)
                one_possibility = current_output[-1, one_index]
                print("one_possibility: ", one_possibility)


                if zero_possibility > one_possibility:
                    relation_extraction = True
                    next_input = "0 . <|endoftext|>"
                    input_ids = torch.cat((input_ids, torch.tensor(tokenizer.encode(next_input, add_special_tokens=False, return_tensors="pt")[0]).to("cuda")), dim=0)
                    continue
                else:
                    relation_extraction = True
                    next_input = "1 . [learn3] [learn4]"
                    input_ids = torch.cat((input_ids, torch.tensor(tokenizer.encode(next_input, add_special_tokens=False, return_tensors="pt")[0]).to("cuda")), dim=0)
                    input_ids = and_the_entities_related(input_index, input_ids.to("cpu"))
                    input_ids = input_ids.to("cuda")
                    # print(tokenizer.decode(input_ids[start:]))
                    continue

            # if relation_extraction:
            #     print(tokenizer.decode(input_ids[start:]))
                
            input_ids = torch.cat((input_ids, torch.tensor(max_index).unsqueeze(0).to("cuda")), dim=0)

        
        print(tokenizer.decode(input_ids[start:]))
        output_texts.append(tokenizer.decode(input_ids[start:]))
        # break
        
    # print(tokenizer.batch_decode(max_index, skip_special_tokens=False)[0])

1  / 96
zero_possibility:  1.4824032
one_possibility:  11.195415


/tmp/ipykernel_237402/3160266930.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.cat((input_ids, torch.tensor(tokenizer.encode(next_input, add_special_tokens=False, return_tensors="pt")[0]).to("cuda")), dim=0)


 for the relation applies to jurisdiction : 1. [learn3] [learn4] and the entities related to the relation applies to jurisdiction are : islation, type : location ; the university of hornia, type : organization. [learn5] [learn6] the entity paris are : head entity: the university of hornia, tail entity: the university of hoenix.<|endoftext|>
2  / 96
zero_possibility:  -6.825655
one_possibility:  -3.3960395
 for the relation author : 1. [learn3] [learn4] and the entities related to the relation author are : ivanhoe gooden, type : head of government ; the city of hanoi, type : location. [learn5] [learn6] the entity paris are : head entity: the city of hanoi, tail entity:ivanhoe gooden; head entity: the city of hanoi, tail entity: johninson.<|endoftext|>
3  / 96
zero_possibility:  0.6180344
one_possibility:  7.798221
 for the relation award received : 1. [learn3] [learn4] and the entities related to the relation award received are : ivan john gooden, type : head of government ; best rap / 

In [81]:
import json

from datasets import Dataset

ner = 1

test_relation_dict = {}
if ner:
    with open('DocRED/data/bi-sent-pre-process_test.json') as f:
        test_relation_dict = json.load(f)

    test_dataset = Dataset.from_dict(
        {
            'text': test_relation_dict['text'],
            'entity': test_relation_dict['entity'],
            'relation': test_relation_dict['relation']
        }
    )

else:
    pass


In [82]:
# save the output_texts to a json file

with open(f'DocRED/GPT_w_ner_short_relation/test_output_texts_for_test_dataset_{random_test_index}.json', 'w') as f:
    json.dump(output_texts, f)

In [83]:
rel_info = {}
with open('DocRED/data/relation-index.json') as f:
    rel_info = json.load(f)
rel_info_list = [relation for relation in rel_info.keys()]

In [84]:
test_dataset[random_test_index]

{'text': 'conrad oberon johnson ( november 15 , 1915 – february 3 , 2008 ) was an american music educator , long associated with the city of houston , who was inducted into the texas bandmasters hall of fame in 2000 . born in victoria , texas , conrad johnson was nine when his family moved to houston .',
 'entity': [['conrad oberon johnson', 'head of government'],
  ['november 15 , 1915', 'time'],
  ['february 3 , 2008', 'time'],
  ['american', 'location'],
  ['houston', 'location'],
  ['texas bandmasters hall of fame', 'organization'],
  ['2000', 'time'],
  ['victoria', 'location'],
  ['texas', 'location'],
  ['conrad johnson', 'head of government'],
  ['nine', 'number']],
 'relation': {'applies to jurisdiction': None,
  'author': None,
  'award received': None,
  'basin country': None,
  'capital': None,
  'capital of': None,
  'cast member': None,
  'chairperson': None,
  'characters': None,
  'child': None,
  'composer': None,
  'conflict': None,
  'contains administrative territor

In [85]:
output_texts[0]

' for the relation applies to jurisdiction : 1. [learn3] [learn4] and the entities related to the relation applies to jurisdiction are : islation, type : location ; the university of hornia, type : organization. [learn5] [learn6] the entity paris are : head entity: the university of hornia, tail entity: the university of hoenix.<|endoftext|>'

In [86]:
right_relation_1 = []
right_relation_0 = []
wrong_relation_1 = []
wrong_relation_0 = []
entity_pairs = []

# text = output_texts[0]

# rel_index = 0

for rel_index in range(len(rel_info_list)):
    text = output_texts[rel_index]
    relation_text = "for the relation " + rel_info_list[rel_index].lower() + " : "
    relation_classfication = text.split(relation_text)[1].split(".")[0].strip()

    if "1" in relation_classfication:
        if test_dataset[random_test_index]['relation'][rel_info_list[rel_index]]:
            print("relation : ", rel_info_list[rel_index])
            print("gold truth: ", test_dataset[random_test_index]['relation'][rel_info_list[rel_index]])
            right_relation_1.append(rel_index)

            entity_pair_for_this_relation = []
            remain_text = text
            while("tail entity" in remain_text):
                if ";" in remain_text.split("tail entity")[1]:
                    head_entity = remain_text.split("head entity: ")[1].split(", tail entity: ")[0].strip()
                    tail_entity = remain_text.split("tail entity: ")[1].split(";")[0].strip()
                    if [head_entity, tail_entity] not in entity_pair_for_this_relation:
                        entity_pair_for_this_relation.append([head_entity, tail_entity])
                        print("head entity: ", head_entity)
                        print("tail entity: ", tail_entity)
                    remain_text = remain_text.split(";")[1].strip()
                else:
                    head_entity = remain_text.split("head entity: ")[1].split(", tail entity: ")[0].strip()
                    tail_entity = remain_text.split("tail entity: ")[1].split(".<|endoftext|>")[0].strip()
                    if [head_entity, tail_entity] not in entity_pair_for_this_relation:
                        entity_pair_for_this_relation.append([head_entity, tail_entity])
                        print("head entity: ", head_entity)
                        print("tail entity: ", tail_entity)
                        print("=====================================")
                    break
            entity_pairs.append(entity_pair_for_this_relation)

        else:
            wrong_relation_1.append(rel_index)
    else:
        if test_dataset[random_test_index]['relation'][rel_info_list[rel_index]]:
            wrong_relation_0.append(rel_index)
        else:
            right_relation_0.append(rel_index)

relation :  contains administrative territorial entity
gold truth:  [['texas', 'victoria'], ['american', 'victoria'], ['american', 'texas']]
head entity:  houston
tail entity:  houston
relation :  country
gold truth:  [['houston', 'american'], ['houston', 'american'], ['texas bandmasters hall of fame', 'american'], ['texas', 'american']]
head entity:  vern raburn
tail entity:  australia
relation :  country of citizenship
gold truth:  [['conrad oberon johnson', 'american'], ['conrad johnson', 'american']]
head entity:  ivan johnstad
tail entity:  australian
relation :  date of birth
gold truth:  [['conrad oberon johnson', 'november 15 , 1915'], ['conrad johnson', 'november 15 , 1915']]
head entity:  ivan johninson
tail entity:  february 3, 1932
relation :  date of death
gold truth:  [['conrad oberon johnson', 'february 3 , 2008'], ['conrad johnson', 'february 3 , 2008']]
head entity:  ivan johninson
tail entity:  february 3, 2009
relation :  located in the administrative territorial ent

In [87]:
print("right_relation_1: ", len(right_relation_1))
print("right_relation_0: ", len(right_relation_0))
print("wrong_relation_1: ", len(wrong_relation_1))
print("wrong_relation_0: ", len(wrong_relation_0))

right_relation_1:  7
right_relation_0:  0
wrong_relation_1:  89
wrong_relation_0:  0


In [88]:
entity_pairs

[[['houston', 'houston']],
 [['vern raburn', 'australia']],
 [['ivan johnstad', 'australian']],
 [['ivan johninson', 'february 3, 1932']],
 [['ivan johninson', 'february 3, 2009']],
 [['australia', 'australia']],
 [['ivan johninson', 'houston']]]

play ground

In [139]:
relation_index = rel_info["date of death"]

In [140]:
tokenizer.convert_tokens_to_ids("[learn5]")

50263

In [141]:
# find the index of 50262 in input_ids_lists[12]

tokenizer.decode(input_ids_lists[relation_index][:input_ids_lists[relation_index].index(50263) + 2])

'conrad oberon johnson ( november 15, 1915 – february 3, 2008 ) was an american music educator, long associated with the city of houston, who was inducted into the texas bandmasters hall of fame in 2000. born in victoria, texas, conrad johnson was nine when his family moved to houston. [learn1] [learn2] for the relation date of death : 1. [learn3] [learn4] and the entities related to the relation date of death are : conrad oberon johnson, type : head of government ; february 3, 2008, type : time ; conrad johnson, type : head of government. [learn5] [learn6]'

In [200]:
print(" for the relation date of death : 1. [learn3] [learn4] and the entities related to the relation date of death are : ivan johninson, type : head of government ; february 3, 2009, type : time. [learn5] [learn6] the entity paris are : head entity: ivan johninson, tail entity: february 3, 2009.<|endoftext|>")

 for the relation date of death : 1. [learn3] [learn4] and the entities related to the relation date of death are : ivan johninson, type : head of government ; february 3, 2009, type : time. [learn5] [learn6] the entity paris are : head entity: ivan johninson, tail entity: february 3, 2009.<|endoftext|>


In [215]:
# input_ids = torch.tensor(input_ids_lists[relation_index][:input_ids_lists[relation_index].index(50263) + 2]).to("cuda")
# input_ids = torch.tensor(input_ids_lists[relation_index][:input_ids_lists[relation_index].index(50263) + 2]).to("cuda")
input_ids = tokenizer.encode("conrad oberon johnson ( november 15, 1915 – february 3, 2008 ) was an american music educator, long associated with the city of houston, who was inducted into the texas bandmasters hall of fame in 2000. born in victoria, texas, conrad johnson was nine when his family moved to houston. [learn1] [learn2] for the relation date of death : 1. [learn5][learn6] the entity paris are : head entity: ", add_special_tokens=False, return_tensors="pt")[0]
input_ids = input_ids.to("cuda")

In [216]:
with torch.no_grad():
    output = model(input_ids=input_ids)
    current_output = np.array(output['logits'].cpu())
    max_index = np.argmax(current_output[-1, :], axis=0)
    sorted_index = np.argsort(current_output[-1, :])[::-1]
    print(tokenizer.decode(max_index))

ivan


In [217]:
def if_in_the_input(token, tokenizer=tokenizer, relation_index=relation_index, input_ids_lists=input_ids_lists):
    learn1_id = tokenizer.convert_tokens_to_ids("[learn1]")
    if token in input_ids_lists[relation_index][:input_ids_lists[relation_index].index(learn1_id)]:
        return "True"
    else:
        return "False"
print("if the token is in the input: ", if_in_the_input(max_index))

if the token is in the input:  False


In [224]:
possible_token = sorted_index[7]
print(tokenizer.decode(possible_token))
print("if the token is in the input: ", if_in_the_input(possible_token))

ik
if the token is in the input:  False


In [213]:
with torch.no_grad():
    input_ids = torch.cat((input_ids, torch.tensor(sorted_index[1]).unsqueeze(0).to("cuda")), dim=0)
    print(tokenizer.decode(input_ids))

conrad oberon johnson ( november 15, 1915 – february 3, 2008 ) was an american music educator, long associated with the city of houston, who was inducted into the texas bandmasters hall of fame in 2000. born in victoria, texas, conrad johnson was nine when his family moved to houston. [learn1] [learn2] for the relation date of death : 1. [learn5] [learn6] the entity related part :
